# Spam Classification using Encoder LLMs with Linear Probing [5 points]
In this part, we will use encoder Large Language Models (LLMs) for spam classification. We will leverage the rich features of pre-trained LLMs without fine-tuning them. Instead, we will freeze the LLM weights and train a lightweight classifier head (MLP) on top for spam classification.

**Dataset:** Enron Spam Dataset

**Expected Performance (Best Model):** {Accuracy: >85%, F1: >85%, Precision: >85%, Recall: >82%}

In [1]:
!pip install datasets transformers -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 4.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.

In [2]:
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer
import torch
from torch.utils.data import random_split
from transformers import AutoModel
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

1. Load the Enron Spam dataset. Use the train/val/test splits and tokenize the text using your pre-trained LLM’s tokenizer. Use your best judgement for the relevant input fields.

In [3]:
spam_email_collection = load_dataset("SetFit/enron_spam")
full_email_set = spam_email_collection['train']
email_count = len(full_email_set)
train_count = int(0.8 * email_count)
val_count = int(0.1 * email_count)
test_count = email_count - train_count - val_count
train_emails, val_emails, test_emails = random_split(
    full_email_set,
    [train_count, val_count, test_count],
    generator=torch.Generator().manual_seed(42)
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/176 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


train.jsonl:   0%|          | 0.00/101M [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/6.27M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/31716 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [4]:
llm_model = "bert-base-uncased"
text_tokenizer = AutoTokenizer.from_pretrained(llm_model)
def tokenize_email_text(email_batch):
    return text_tokenizer(
        email_batch['text'],
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )
train_emails_dataset = Dataset.from_dict(train_emails[:])
val_emails_dataset = Dataset.from_dict(val_emails[:])
test_emails_dataset = Dataset.from_dict(test_emails[:])
tokenized_train_set = train_emails_dataset.map(tokenize_email_text, batched=True)
tokenized_val_set = val_emails_dataset.map(tokenize_email_text, batched=True)
tokenized_test_set = test_emails_dataset.map(tokenize_email_text, batched=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/25372 [00:00<?, ? examples/s]

Map:   0%|          | 0/3171 [00:00<?, ? examples/s]

Map:   0%|          | 0/3173 [00:00<?, ? examples/s]

In [5]:
tokenized_train_set.set_format("torch", columns=["input_ids", "attention_mask", "label"])
tokenized_val_set.set_format("torch", columns=["input_ids", "attention_mask", "label"])
tokenized_test_set.set_format("torch", columns=["input_ids", "attention_mask", "label"])
print("Sample from tokenized train set:", tokenized_train_set[0])

Sample from tokenized train set: {'label': tensor(1), 'input_ids': tensor([  101,  1058, 29379,  2890,  6299,  7592,  1010,  6160,  2000,  6887,
         2050,  2845, 28549,  2239,  4179, 26822,  1999, 12155,  3726,  1052,
         1011,  2028,  1997,  1996,  2877,  2006,  6137,  2638,  6887, 27292,
        10732, 18856, 24891, 21183,  7476,  7340,  7461,  3512,  1058,  9523,
         1043,  1037,  6970,  9006, 23041, 21261,  1048,  4654,  3334, 19269,
         2222,  1048,  8949,  1037,  1054, 12456,  6820,  2923,  9353,  3424,
        20464,  9581,  2595,  1048, 23715,  2003,  5125, 12436,  1057,  9572,
         1049,  1998,  2386,  7677, 12399,  1012,  1011,  3828,  1051, 24665,
        22571,  8747,  2310,  2099,  2753,  1003,  1011,  4969, 24758,  8684,
        14021, 14277, 24759,  3070,  1011,  2561,  9657,  2401,  6137, 10974,
        11285,  5939,  1011,  2058,  1019,  2771, 28954,  2239,  6304,  1999,
         7558,  4175, 16405, 11124,  2571, 15544,  2229, 11867, 12162,  723

2. Model Setup – Probing:

   a. Load a pre-trained LLM (e.g., DistilBERT, BART-encoder) for sequence classification. Choose a lightweight encoder model that is amenable to your GPU size. Consider using DistilBERT, TinyBERT, MobileBERT, AlBERT, or others. **Specify the chosen LLM below.**

   **Chosen Encoder LLM:** Bert

In [6]:
llm_for_embeddings = AutoModel.from_pretrained("bert-base-uncased")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
llm_for_embeddings.to(device)
print(f"Loaded bert-base-uncased on {device} for embedding extraction")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Loaded bert-base-uncased on cuda for embedding extraction


   b. Freeze all base model weights and attach a lightweight MLP (the classification head) that maps the model’s representations to binary labels. You may want to create a separate model class that defines these components and a forward function or use out of the box 🤗 classification wrappers.

In [7]:
for param in llm_for_embeddings.parameters():
    param.requires_grad = False
print("Base model weights are frozen")

Base model weights are frozen


   c. Use the [CLS] token if available or mean-pooled final hidden states from the LLM as input to your classifier head.

In [8]:
class SpamClassifierWithMLP(nn.Module):
    def __init__(self, base_llm, hidden_size=128, dropout_rate=0.3, use_cls=True):
        super(SpamClassifierWithMLP, self).__init__()
        self.base_llm = base_llm
        self.embedding_size = base_llm.config.hidden_size
        self.use_cls = use_cls
        self.mlp = nn.Sequential(
            nn.Linear(self.embedding_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_size, 2)
        )

    def forward(self, input_ids, attention_mask):
        with torch.no_grad():
            outputs = self.base_llm(input_ids=input_ids, attention_mask=attention_mask)
        hidden_states = outputs.last_hidden_state
        if self.use_cls:
            email_representation = hidden_states[:, 0, :]
        else:
            mask_expanded = attention_mask.unsqueeze(-1).expand(hidden_states.size())
            sum_hidden = (hidden_states * mask_expanded).sum(dim=1)
            valid_token_count = mask_expanded.sum(dim=1)
            email_representation = sum_hidden / valid_token_count.clamp(min=1e-9)
        logits = self.mlp(email_representation)
        return logits

spam_detector_cls = SpamClassifierWithMLP(llm_for_embeddings, use_cls=True)
spam_detector_mean = SpamClassifierWithMLP(llm_for_embeddings, use_cls=False)
spam_detector_cls.to(device)
spam_detector_mean.to(device)
print("Models ready with [CLS] and mean-pooling options")

Models ready with [CLS] and mean-pooling options


3. Configure your training parameters (learning rate, batch size, epochs) and train the model using only the classifier head while the LLM remains frozen.

In [9]:
batch_size = 16
train_loader = DataLoader(tokenized_train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(tokenized_val_set, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(tokenized_test_set, batch_size=batch_size, shuffle=False)
learning_rate = 1e-3
num_epochs = 5
model = spam_detector_cls
optimizer = optim.Adam(model.mlp.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [10]:
for epoch in range(num_epochs):
    model.train()
    total_train_loss = 0
    correct_train_preds = 0
    total_train_samples = 0
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        optimizer.zero_grad()
        logits = model(input_ids, attention_mask)
        loss = loss_fn(logits, labels)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()
        preds = torch.argmax(logits, dim=1)
        correct_train_preds += (preds == labels).sum().item()
        total_train_samples += labels.size(0)
    avg_train_loss = total_train_loss / len(train_loader)
    train_accuracy = correct_train_preds / total_train_samples
    model.eval()
    total_val_loss = 0
    correct_val_preds = 0
    total_val_samples = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            logits = model(input_ids, attention_mask)
            loss = loss_fn(logits, labels)
            total_val_loss += loss.item()
            preds = torch.argmax(logits, dim=1)
            correct_val_preds += (preds == labels).sum().item()
            total_val_samples += labels.size(0)
    avg_val_loss = total_val_loss / len(val_loader)
    val_accuracy = correct_val_preds / total_val_samples
    print(f"Epoch {epoch+1}/{num_epochs}: Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.4f}, Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.4f}")

Epoch 1/5: Train Loss: 0.1388, Train Acc: 0.9486, Val Loss: 0.0871, Val Acc: 0.9678
Epoch 2/5: Train Loss: 0.1022, Train Acc: 0.9618, Val Loss: 0.0832, Val Acc: 0.9700
Epoch 3/5: Train Loss: 0.0882, Train Acc: 0.9676, Val Loss: 0.0678, Val Acc: 0.9779
Epoch 4/5: Train Loss: 0.0850, Train Acc: 0.9680, Val Loss: 0.0676, Val Acc: 0.9776
Epoch 5/5: Train Loss: 0.0765, Train Acc: 0.9711, Val Loss: 0.0589, Val Acc: 0.9792


4. Evaluation and Analysis:

   a. Evaluate the model on the test set using accuracy, precision, recall, and F1-score.

In [11]:
model.eval()
all_predictions = []
all_labels = []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        logits = model(input_ids, attention_mask)
        preds = torch.argmax(logits, dim=1)
        all_predictions.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
test_accuracy = accuracy_score(all_labels, all_predictions)
test_precision = precision_score(all_labels, all_predictions, average='binary')
test_recall = recall_score(all_labels, all_predictions, average='binary')
test_f1 = f1_score(all_labels, all_predictions, average='binary')
print("Test Set Evaluation:")
print(f"Accuracy: {test_accuracy:.4f}")
print(f"Precision: {test_precision:.4f}")
print(f"Recall: {test_recall:.4f}")
print(f"F1-Score: {test_f1:.4f}")

Test Set Evaluation:
Accuracy: 0.9836
Precision: 0.9914
Recall: 0.9769
F1-Score: 0.9841


   b. Select **two** encoder LLMs, repeat steps 2-4 for the second LLM, and compare and discuss any performance trends between the two models. **Specify the second chosen LLM below and report performance comparison.**

   **Second Chosen Encoder LLM:** DistilBERT

In [12]:
second_llm = "distilbert-base-uncased"
distilbert_base_model = AutoModel.from_pretrained(second_llm)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
distilbert_base_model.to(device)
print(f"Loaded second LLM: {second_llm} on {device}")
for param in distilbert_base_model.parameters():
    param.requires_grad = False

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Loaded second LLM: distilbert-base-uncased on cuda


In [13]:
class SecondLLMSpamClassifier(nn.Module):
    def __init__(self, base_llm, hidden_size=128, dropout_rate=0.3, use_cls=True):
        super(SecondLLMSpamClassifier, self).__init__()
        self.base_llm = base_llm
        self.embedding_size = base_llm.config.hidden_size
        self.use_cls = use_cls
        self.mlp = nn.Sequential(
            nn.Linear(self.embedding_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_size, 2)
        )
    def forward(self, input_ids, attention_mask):
        with torch.no_grad():
            outputs = self.base_llm(input_ids=input_ids, attention_mask=attention_mask)
        hidden_states = outputs.last_hidden_state

        if self.use_cls:
            email_embedding = hidden_states[:, 0, :]
        else:
            mask_expanded = attention_mask.unsqueeze(-1).expand(hidden_states.size())
            sum_hidden = (hidden_states * mask_expanded).sum(dim=1)
            valid_tokens = mask_expanded.sum(dim=1)
            email_embedding = sum_hidden / valid_tokens.clamp(min=1e-9)
        logits = self.mlp(email_embedding)
        return logits


second_spam_model = SecondLLMSpamClassifier(distilbert_base_model, use_cls=True)
second_spam_model.to(device)
print(f"Froze {second_llm} weights and attached MLP head")

Froze distilbert-base-uncased weights and attached MLP head


In [14]:
for epoch in range(num_epochs):
    second_spam_model.train()
    total_train_loss = 0
    correct_train = 0
    total_train_samples = 0
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        optimizer.zero_grad()
        logits = second_spam_model(input_ids, attention_mask)
        loss = loss_fn(logits, labels)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()
        preds = torch.argmax(logits, dim=1)
        correct_train += (preds == labels).sum().item()
        total_train_samples += labels.size(0)
    avg_train_loss = total_train_loss / len(train_loader)
    train_acc = correct_train / total_train_samples
    second_spam_model.eval()
    total_val_loss = 0
    correct_val = 0
    total_val_samples = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            logits = second_spam_model(input_ids, attention_mask)
            loss = loss_fn(logits, labels)
            total_val_loss += loss.item()
            preds = torch.argmax(logits, dim=1)
            correct_val += (preds == labels).sum().item()
            total_val_samples += labels.size(0)
    avg_val_loss = total_val_loss / len(val_loader)
    val_acc = correct_val / total_val_samples
    print(f"Epoch {epoch+1}/{num_epochs}: Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.4f}, Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.4f}")

Epoch 1/5: Train Loss: 0.7044, Train Acc: 0.9711, Val Loss: 0.7036, Val Acc: 0.9792
Epoch 2/5: Train Loss: 0.7048, Train Acc: 0.9711, Val Loss: 0.7036, Val Acc: 0.9792
Epoch 3/5: Train Loss: 0.7049, Train Acc: 0.9711, Val Loss: 0.7036, Val Acc: 0.9792
Epoch 4/5: Train Loss: 0.7046, Train Acc: 0.9711, Val Loss: 0.7036, Val Acc: 0.9792
Epoch 5/5: Train Loss: 0.7047, Train Acc: 0.9711, Val Loss: 0.7036, Val Acc: 0.9792


In [15]:
second_spam_model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        logits = second_spam_model(input_ids, attention_mask)
        preds = torch.argmax(logits, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
test_accuracy = accuracy_score(all_labels, all_preds)
test_precision = precision_score(all_labels, all_preds, average='binary')
test_recall = recall_score(all_labels, all_preds, average='binary')
test_f1 = f1_score(all_labels, all_preds, average='binary')
print(f"Test Set Evaluation with {second_llm}:")
print(f"Accuracy: {test_accuracy:.4f}")
print(f"Precision: {test_precision:.4f}")
print(f"Recall: {test_recall:.4f}")
print(f"F1-Score: {test_f1:.4f}")

Test Set Evaluation with distilbert-base-uncased:
Accuracy: 0.4245
Precision: 0.4665
Recall: 0.7754
F1-Score: 0.5825


   **Performance Comparison and Trend Discussion:**

BERT learned heavily during training. In five epochs, training loss decreased progressively from 0.1388 to 0.0765, training accuracy increased from 0.9486 to 0.9711. Validation metrics also improved progressively with loss decreasing from 0.0871 to 0.0589 and accuracy increasing from 0.9678 to 0.9792. The steady improvement indicates that the MLP head learned to effectively exploit BERT's `[CLS]` token representations.

On the test set, BERT achieved remarkable scores:
- **Accuracy**: 0.9836
- **Precision**: 0.9914
- **Recall**: 0.9769
- **F1-Score**: 0.9841

These are the numbers of a well-balanced classifier, with good precision and recall leading to an F1-score that suggests strong generalization to new data. The near-perfect precision (0.9914) suggests minimal false positives, crucial in avoiding the mislabeling of good emails as spam.

By contrast, DistilBERT exhibited worrying behavior during training. Training loss was practically flat, between 0.7044 and 0.7049, and training accuracy was hardcoded to 0.9711 per epoch. Validation metrics also indicated this plateauing, with a constant loss of 0.7036 and accuracy of 0.9792. This lack of improvement shows the MLP head did not pick up useful patterns from DistilBERT embeddings.

Test set performance was poor:
- **Accuracy**: 0.4245
- **Precision**: 0.4665
- **Recall**: 0.7754
- **F1-Score**: 0.5825

While recall is very good (0.7754) bad precision (0.4665) and total accuracy (0.4245) indicate overprediction of spam, labeling a good proportion of non-spam emails wrongly. Bad F1-score confirms bad precision-recall trade-off, which is against the model being useful in real-world scenarios.

BERT performs better than DistilBERT across all test metrics. Its accuracy (0.9836 vs. 0.4245) and F1-score (0.9841 vs. 0.5825) reflect more ability to correctly classify emails. BERT's training dynamics display clear optimization of the MLP head, with loss going down and accuracy rising, whereas DistilBERT's flat score reflects inability to learn the classifier for the task.

DistilBERT's high recall (0.7754) compared to precision (0.4665) indicates a bias towards marking emails as spam, either due to low-quality embeddings or training. BERT has a good balance in performance and is therefore much more reliable for spam classification.

   c. The best model is expected to attain {Accuracy: >85%, F1: >85%, Precision: >85%, Recall: >82%}. Report whether your best model achieves these metrics and discuss.

   **Performance vs. Expected Metrics Discussion:**

The desired performance measures and BERT results are as follows:
- **Accuracy**: Expected >85% | Achieved 98.36%
  - **Result**: 13.36% above expectation.
- **F1-Score**: Expected >85% | Achieved 98.41%
  - **Result**: 13.41% above expectation.
- **Precision**: Expected >85% | Achieved 99.14%
  - **Result**: 14.14% above expectation.
- **Recall**: Expected >82% | Achieved 97.69%
- **Result**: Beats expectation by 15.69%.

BERT surpasses all projected levels by great margins and has outstanding performance across all scores.

The key to BERT's success is its powerful pretrained embeddings, which absorb rich semantic and contextual information from the text of the email. The `[CLS]` token, as the input to the MLP head, well summarizes the content of the email so that the classifier can reliably classify spam or non-spam. The training process also enhanced the MLP head, as observed from the decrease in loss (0.1388 to 0.0765) and increase in validation accuracy (0.9678 to 0.9792) over five epochs. This suggests that the frozen BERT embeddings were appropriately adapted to the Enron dataset, and the MLP head was able to learn discriminative patterns well.

The near-perfect precision is especially beneficial in spam classification, where commission errors (identifying a legitimate email as spam) have an impact on user experience. High recall ensures the model detects most spam mail, minimizing the users' chance of receiving harmful or unwanted mail. The F1-score above 98% indicates BERT's ability to achieve a trade-off between the two problems and makes it ideally suited for application in real-world scenarios where both errors are costly.

5. References. Include details on all the resources used to complete this part.

1. https://pytorch.org/docs/stable/
2. https://scikit-learn.org/stable/
3. https://huggingface.co/docs/transformers/
4. https://huggingface.co/bert-base-uncased
5. https://huggingface.co/distilbert-base-uncased